In [ ]:
import numpy as np
import pandas as pd
import string


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/dataset/data.csv')
tempo = np.array(dataset[['text','label_num']])

X = np.array(tempo[:,0])
y = np.array(tempo[:,-1]).astype('float32')

In [ ]:
#lower case
#removing Punctuation
transTable = str.maketrans(dict.fromkeys(string.punctuation))
X = np.array([sen.lower().translate(transTable) for sen in X])

import re
def remove_hyperlink(word):
    return re.sub(r"http\S+", "", word)

X = np.array([remove_hyperlink(sen) for sen in X])

In [ ]:
max_feature = 50000
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_feature)
tokenizer.fit_on_texts(X)
X_features =  np.array(tokenizer.texts_to_sequences(X))

max_len = 0;
for arr in X_features:
    max_len = max(max_len,len(arr))

from keras.preprocessing.sequence import pad_sequences
X_features = pad_sequences(X_features,maxlen=max_len,padding='post')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [ ]:
X_features = X_features.astype('float32')

print(X_features)

[[9.0000e+00 1.7000e+01 1.0420e+03 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [9.0000e+00 4.7000e+01 1.4700e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [9.0000e+00 1.4390e+03 2.7143e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [9.0000e+00 7.0300e+02 2.4000e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [9.0000e+00 1.0630e+03 9.4280e+03 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [9.0000e+00 9.0600e+02 3.2900e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]]


In [ ]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X_features, y, test_size=0.33, random_state=42,shuffle=True)
X_test, X_valid, y_test, y_valid = tts(X_test, y_test, test_size=0.5, random_state=42,shuffle=True)


In [ ]:
X_features.shape

(5171, 3955)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import RandomFourierFeatures

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(3955,)),
        RandomFourierFeatures(output_dim=5000,scale=10.0,kernel_initializer='gaussian',trainable=True),
        layers.Dense(20, activation='relu'),
        layers.Dropout(0.33),
        layers.Dense(1, activation='sigmoid')
    ]
)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_fourier_features_10 ( (None, 5000)              19780001  
_________________________________________________________________
dense_22 (Dense)             (None, 20)                100020    
_________________________________________________________________
dropout_11 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 21        
Total params: 19,880,042
Trainable params: 100,042
Non-trainable params: 19,780,000
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train,y_train,batch_size = 500,epochs=20,validation_data=(X_valid,y_valid),verbose=True)

Epoch 1/20
7/7 [==============================] - 7s 987ms/step - loss: 0.7198 - accuracy: 0.5773 - val_loss: 0.6691 - val_accuracy: 0.6019
Epoch 2/20
7/7 [==============================] - 6s 848ms/step - loss: 0.6306 - accuracy: 0.6616 - val_loss: 0.6491 - val_accuracy: 0.6241
Epoch 3/20
7/7 [==============================] - 6s 842ms/step - loss: 0.5361 - accuracy: 0.7416 - val_loss: 0.6526 - val_accuracy: 0.6393
Epoch 4/20
7/7 [==============================] - 6s 817ms/step - loss: 0.4687 - accuracy: 0.7872 - val_loss: 0.6606 - val_accuracy: 0.6300
Epoch 5/20
7/7 [==============================] - 6s 836ms/step - loss: 0.4185 - accuracy: 0.8219 - val_loss: 0.6613 - val_accuracy: 0.6253
Epoch 6/20
7/7 [==============================] - 6s 822ms/step - loss: 0.4429 - accuracy: 0.8069 - val_loss: 0.6604 - val_accuracy: 0.6393
Epoch 7/20
7/7 [==============================] - 6s 842ms/step - loss: 0.3465 - accuracy: 0.8739 - val_loss: 0.6806 - val_accuracy: 0.6218
Epoch 8/20
7/7 [====

In [ ]:
model.save("/content/drive/MyDrive/spam-detector/svm.model",save_format="h5")